# Replication Code (Python): Age, Income, and the Discounting of Delayed Monetary Losses

This Jupyter Notebook provides a complete Python-based replication of the analyses from the publication:

> Wan, H., Myerson, J., Green, L., Strube, M. J., & Hale, S. (2025). Age, income, and the discounting of delayed monetary losses. *The Journals of Gerontology, Series B: Psychological Sciences and Social Sciences, 80*(11), gbaf162. https://doi.org/10.1093/geronb/gbaf162

The original analysis, conducted in R, has been translated into a Python workflow. This notebook demonstrates proficiency in replicating complex statistical analyses across different programming ecosystems.

### Workflow Overview

1.  **Setup**: Imports all necessary Python libraries and defines custom functions for modeling and plotting.
2.  **Data Processing**: Loads the raw data and cleans it using `pandas`, preparing it for analysis.
3.  **Descriptive & Correlational Analyses**: Replicates the paper's descriptive statistics, visualizations (Figure 1), and correlation tables using `matplotlib`, `seaborn`, and `pingouin`.
4.  **Hypothesis Testing**: Fits the main Bayesian multilevel beta regression models from the paper (Table 3) using `pymc`.
5.  **Supplementary Analyses**: Fits additional models to test the roles of depression and overall distress.
6.  **Interaction Plots**: Generates visualizations for the key model interactions (Figure 4), demonstrating the ability to interpret and communicate complex model findings.

The data for this study are available in the Supplementary Material at the publisher's website.

In [ ]:
# --- 1. Environment Setup ---
# This cell installs all the required Python packages.
# It is commented out by default. Uncomment and run this cell if the packages are not yet installed in your environment.

# import sys
# !{sys.executable} -m pip install pandas numpy scipy statsmodels pymc arviz matplotlib seaborn pingouin patsy jax jaxlib

In [ ]:
# --- 2. Imports and Custom Functions ---

# --- Core Libraries ---
import sys
import pandas as pd
import numpy as np
import warnings
import patsy


# --- Statistics and Modeling ---
from sklearn.metrics import auc as calculate_auc, r2_score
from scipy.optimize import curve_fit
import pymc as pm
import arviz as az
import pingouin as pg

# --- Visualization ---
import matplotlib.pyplot as plt
import seaborn as sns

# --- Global Settings ---
# Suppress warnings for a cleaner notebook output
warnings.filterwarnings('ignore')
# Set a consistent plotting style
plt.style.use('seaborn-v0_8-whitegrid')
# Standardize float display format
pd.options.display.float_format = '{:.3f}'.format


def set_mattheme(ax, title="", xlabel="", ylabel=""):
    """
    Applies a consistent, publication-quality theme to a Matplotlib axes object.
    
    Args:
        ax (matplotlib.axes.Axes): The axes object to style.
        title (str): The title for the plot.
        xlabel (str): The label for the x-axis.
        ylabel (str): The label for the y-axis.
    """
    ax.set_title(title, fontsize=14, fontweight='bold', pad=15)
    ax.set_xlabel(xlabel, fontsize=12, fontweight='bold', labelpad=10)
    ax.set_ylabel(ylabel, fontsize=12, fontweight='bold', labelpad=10)
    for spine in ax.spines.values():
        spine.set_edgecolor('black')
        spine.set_linewidth(1)
    ax.set_facecolor('white')
    ax.grid(False)
    ax.tick_params(axis='both', which='major', labelsize=10, direction='out', width=1, length=5)
    ax.legend(frameon=False)


# --- betaMLM_pymc function ---
def betaMLM_pymc(formula, data):
    """
    Fits a Bayesian multilevel beta regression model using PyMC.

    This function uses `patsy` to parse the model formula, constructs the
    model in PyMC with weakly informative priors, and samples from the posterior.

    Args:
        formula (str): A patsy-style formula for the model (e.g., "y ~ x1 + (1|group)").
        data (pd.DataFrame): The dataframe containing the model variables.

    Returns:
        arviz.InferenceData: The fitted model object containing posterior samples.
    """
    # 1. Prepare coordinates for random effects, if they exist in the formula
    coords = {}
    if '(1|ID)' in formula:
        fixed_formula = formula.split('(1|ID)')[0].strip().rstrip('+').strip()
        has_re = True
        id_idx, id_cats = pd.factorize(data['ID'])
        coords['ID_dim'] = id_cats
    else:
        fixed_formula = formula
        has_re = False

    # 2. Use patsy to create design matrices for the fixed effects
    y_patsy, X_patsy = patsy.dmatrices(fixed_formula, data=data)
    y_data = np.asarray(y_patsy).ravel()
    X_data = np.asarray(X_patsy)
    X_cols = X_patsy.design_info.column_names

    # 3. Construct the PyMC model
    with pm.Model(coords=coords) as model:
        # --- Priors (matching the R brms setup) ---
        intercept = pm.Normal('Intercept', mu=0, sigma=100)
        slopes = pm.Cauchy('slopes', alpha=0, beta=2.5, shape=X_data.shape[1] - 1)
        betas = pm.math.concatenate([[intercept], slopes])
        
        # Priors for random effects (if any)
        if has_re:
            sigma_id = pm.HalfCauchy('sigma_id', beta=2.5)
            z_id = pm.Normal('z_id', mu=0, sigma=1, dims='ID_dim')
            intercept_id = pm.Deterministic('intercept_id', z_id * sigma_id, dims='ID_dim')
        
        # CHANGED: Renamed 'phi' to 'nu' to match PyMC's parameterization.
        nu = pm.HalfNormal('nu', sigma=100)

        # --- Linear Model (eta) ---
        eta = pm.math.dot(X_data, betas)
        if has_re:
            eta += intercept_id[id_idx]

        # --- Likelihood ---
        # Transform eta to the (0, 1) scale for the mean of the Beta distribution
        mu = pm.math.invlogit(eta)
        
        # CHANGED: Used 'nu' instead of 'phi' in the pm.Beta call.
        pm.Beta(y_patsy.design_info.column_names[0], mu=mu, nu=nu, observed=y_data)
        
        # --- Sampling from the Posterior ---
        idata = pm.sample(draws=2000, tune=2000, chains=4, cores=4, progressbar=False, target_accept=0.95)
        
        # Rename slope variables for a cleaner summary table
        idata.posterior = idata.posterior.rename_vars(
            {'slopes': [col for col in X_cols if col != 'Intercept']}
        )
    return idata


def pymc_summary(idata, var_names=None):
    """
    Creates a formatted summary table from a fitted PyMC model.

    Args:
        idata (arviz.InferenceData): The fitted model object.
        var_names (list, optional): A list of variable names to include in the summary.

    Returns:
        pd.DataFrame: A formatted summary table with posterior estimates and diagnostics.
    """
    if var_names is None:
        # Auto-detect fixed-effect variable names
        var_names = [v for v in idata.posterior.data_vars if 'z_id' not in v and 'sigma_id' not in v and 'intercept_id' not in v]

    summary = az.summary(idata, var_names=var_names, kind='stats', hdi_prob=0.95)
    posterior = az.extract(idata, var_names=var_names)
    
    # Calculate Probability of Direction (pd)
    pd_values = {}
    for var in posterior.data_vars:
        samples = posterior[var].values.flatten()
        pd_val = np.mean(samples > 0)
        pd_val = max(pd_val, 1 - pd_val)
        pd_values[var] = pd_val
        
    summary['pd'] = summary.index.map(pd_values).round(3)
    # Add a significance flag based on pd
    summary['signif'] = np.where(summary['pd'] >= 0.975, '*', ' ')
    summary = summary[['mean', 'sd', 'pd', 'signif']]
    summary.columns = ['Est.', '(SE)', 'pd', '']
    summary['Est.(SE)'] = summary.apply(lambda row: f"{row['Est.']:.3f} ({row['(SE)']:.3f})", axis=1)
    
    return summary[['Est.(SE)', 'pd', '']]

In [ ]:
# --- 3. Data Loading and Processing ---

# --- Load Raw Data ---
# The original dataset contains data from multiple studies.
Samp = pd.read_csv("Lifespan.csv", index_col=0) 

# --- Initial Filtering and Cleaning (to match R script) ---
# This chain filters to the final sample (N = 594) used in the paper.
Disc_Raw = Samp[
    (Samp['type'] == 'delay') & (Samp['task'] == 'loss') &
    (Samp['age'].notna()) & (Samp['age'].between(20, 80)) &
    (Samp['check'] == 6) & (Samp['sex'].notna()) &
    ((Samp['age'] - 10 * Samp['age_grp']).between(0, 10)) # Removes inconsistent age reporting
].copy()

# --- Feature Engineering and Renaming ---
# Create Age_Group factor for plotting and descriptive statistics
age_bins = [19, 34, 50, 64, 80]
age_labels = ["20-34", "35-50", "51-64", "65-80"]
Disc_Raw['age_grp_factor'] = pd.cut(Disc_Raw['age'], bins=age_bins, labels=age_labels)

# Rename columns for clarity and consistency with the paper
rename_dict = {
    'type': 'Exp', 'id': 'ID', 'amt': 'Amount', 'iv': 'Delay', 'value': 'SV', 'age': 'Age',
    'income': 'Income', 'sex': 'Gender', 'depress': 'Depression', 'anxious': 'Anxiety',
    'hads': 'Distress', 'edu': 'Education', 'health': 'Health', 'age_grp_factor': 'Age_Group',
    'eth': 'Ethnicity', 'race': 'Race'
}
Disc_Raw = Disc_Raw.rename(columns=rename_dict).filter(items=rename_dict.values())

# --- Create Group-Level Dataframe for Plotting (Figure 1) ---
Disc_Grp = Disc_Raw.copy()
Disc_Grp['Amount_cat'] = pd.Categorical(
    Disc_Grp['Amount'].replace({1: "$150", 2: "$2,500", 3: "$30,000"}),
    categories=["$150", "$2,500", "$30,000"], ordered=True
)
Disc_Grp = Disc_Grp.groupby(['Age_Group', 'Amount_cat', 'Delay'], observed=True).agg(Mean_SV=('SV', 'mean')).reset_index()

# --- Calculate Area under the Curve (AuC) for Each Individual ---
# AuC is the primary dependent variable, calculated for each participant at each loss amount.
AuC_ID = Disc_Raw.sort_values('Delay').groupby(['ID', 'Amount']).apply(
    lambda g: calculate_auc(g['Delay'] / g['Delay'].max(), g['SV'])
).reset_index(name='AuC')

# Join back the unique demographic information for each participant
demographics = Disc_Raw.drop_duplicates(subset='ID').drop(columns=['Amount', 'Delay', 'SV', 'Exp'])
AuC_ID = pd.merge(AuC_ID, demographics, on='ID', how='left')

# Remap Education from codes to years
edu_map = {1: 5, 2: 12, 3: 14, 4: 16, 5: 18}
AuC_ID['Education'] = AuC_ID['Education'].map(edu_map)

# --- Create Model-Specific Dataframes with Scaled Predictors ---
# Predictors are scaled by subtracting the mean and dividing by 2*SD for model stability and interpretability.
def scale_var(series):
    return (series - series.mean()) / (2 * series.std())

# Data for participants aged 35+
DL_dat1 = AuC_ID[AuC_ID['Age'] >= 35].copy()
DL_dat1['Age_std'] = scale_var(DL_dat1['Age'])

# Data for model with Income
DL_dat2 = AuC_ID[(AuC_ID['Age'] >= 35) & AuC_ID['Income'].notna()].copy()
for col in ['Age', 'Income']:
    DL_dat2[f'{col}_std'] = scale_var(DL_dat2[col])

# Data for models with psychological distress measures
DL_dat3 = AuC_ID[(AuC_ID['Age'] >= 35) & AuC_ID['Income'].notna() & AuC_ID['Distress'].notna()].copy()
for col in ['Age', 'Income', 'Anxiety', 'Depression', 'Distress']:
    DL_dat3[f'{col}_std'] = scale_var(DL_dat3[col])

# Data for the full model (Model 4)
DL_dat4 = AuC_ID[
    (AuC_ID['Age'] >= 35) & AuC_ID['Income'].notna() & AuC_ID['Gender'].notna() & 
    AuC_ID['Education'].notna() & AuC_ID['Distress'].notna() & AuC_ID['Health'].notna()
].copy()
for col in ['Age', 'Amount', 'Income', 'Education', 'Anxiety', 'Depression', 'Distress', 'Health']:
    DL_dat4[f'{col}_std'] = scale_var(DL_dat4[col])
DL_dat4['Gender_c'] = DL_dat4['Gender'] - DL_dat4['Gender'].mean() # Center binary predictor

# Clip AuC values for Beta regression, which requires the outcome to be in the open interval (0, 1)
for df in [DL_dat1, DL_dat2, DL_dat3, DL_dat4]:
    df['AuC'] = np.clip(df['AuC'], 1e-5, 1 - 1e-5)

print(f"Data processing complete. Final sample size: {AuC_ID['ID'].nunique()} participants.")

## Descriptive & Correlational Analyses

This section replicates the descriptive results from the paper, including the visualization of group-level discounting functions (Figure 1), goodness-of-fit statistics for the hyperboloid model, reliability of the AuC measure, and the correlation matrices for key variables.

In [ ]:
# --- 4. DESCRIPTIVE ANALYSES ---

# --- Figure 1 Replication: Group-Level Discounting Functions ---
def hyperboloid(delay, k, s):
    """Mathematical function for hyperboloid discounting."""
    return 1 / (1 + k * delay)**s

# --- Goodness-of-Fit for Group-Level Hyperboloid Functions ---
print("\n--- R-squared for Group-Level Hyperboloid Fits ---")
r2_results = Disc_Grp.groupby(['Age_Group', 'Amount_cat'], observed=True).apply(
    lambda grp: r2_score(grp['Mean_SV'], hyperboloid(grp['Delay'], *curve_fit(hyperboloid, grp['Delay'], grp['Mean_SV'], p0=[0.1, 1])[0]))
).unstack()
print(r2_results)

### Reliability and Correlational Analyses

This section assesses the internal consistency of the Area under the Curve (AuC) measure across the three different loss amounts. It also replicates the correlation matrices from the paper for both the full sample (ages 20-80) and the primary analysis sample (ages 35-80).

In [ ]:
# --- Reliability of AuC Measure (Cronbach's Alpha) ---
print("--- Reliability of AuC Measure (Cronbach's Alpha) ---")
reliability_df = AuC_ID.pivot(index='ID', columns='Amount', values='AuC')
print(pg.cronbach_alpha(data=reliability_df))

# --- Correlation Matrix for All Participants (Aged 20-80) ---
print("\n--- Correlation Matrix for All Participants (Aged 20-80) ---")
Cor_df_full = AuC_ID.groupby('ID').agg(
    Age=('Age', 'mean'), Income=('Income', 'mean'), Education=('Education', 'mean'),
    Gender=('Gender', 'mean'), Distress=('Distress', 'mean'), Anxiety=('Anxiety', 'mean'),
    Depression=('Depression', 'mean'), Health=('Health', 'mean'), AuC=('AuC', 'mean')
)
print(Cor_df_full.corr(method='pearson').round(3))

# --- Correlation Matrix for Participants Aged 35-80 (Table 2 Replication) ---
print("\n--- Correlation Matrix for Participants Aged 35-80 ---")
Cor_df_35plus = AuC_ID[AuC_ID['Age'] >= 35].groupby('ID').agg(
    Age=('Age', 'mean'), Income=('Income', 'mean'), Education=('Education', 'mean'),
    Gender=('Gender', 'mean'), Distress=('Distress', 'mean'), Anxiety=('Anxiety', 'mean'),
    Depression=('Depression', 'mean'), Health=('Health', 'mean'), AuC=('AuC', 'mean')
)
print(Cor_df_35plus.corr(method='pearson').round(3))

## Hypothesis Testing: Bayesian Multilevel Models

This section replicates the main hypothesis tests from the paper, corresponding to Table 3. A series of four Bayesian multilevel beta regression models are fitted to test the effects of age, income, anxiety, and other covariates on AuC for participants aged 35 and older.

In [ ]:
# --- 5. HYPOTHESIS TESTING: MAIN MODELS ---

# --- Model 1: AuC ~ Age + Age^2 ---
# Tests for a linear and quadratic effect of age on discounting.
print("--- Model 1: AuC ~ Age + Age^2 ---")
idata_mod1b = betaMLM_pymc("AuC ~ Age_std + I(Age_std**2) + (1|ID)", DL_dat1)
print(pymc_summary(idata_mod1b))

# --- Model 2: AuC ~ Age + Age^2 + Income ---
# Adds income as a predictor.
print("\n--- Model 2: AuC ~ Age + Age^2 + Income ---")
idata_mod2 = betaMLM_pymc("AuC ~ Age_std + I(Age_std**2) + Income_std + (1|ID)", DL_dat2)
print(pymc_summary(idata_mod2))

# --- Model 3: AuC ~ Age + Age^2 + Income + Anxiety ---
# Adds anxiety as a predictor.
print("\n--- Model 3: AuC ~ Age + Age^2 + Income + Anxiety ---")
idata_mod3 = betaMLM_pymc("AuC ~ Age_std + I(Age_std**2) + Income_std + Anxiety_std + (1|ID)", DL_dat3)
print(pymc_summary(idata_mod3))

# --- Model 4: Full Model with Covariates and Age Interactions ---
# The full model includes all covariates and their interactions with age.
print("\n--- Model 4: Full Model with Covariates and Age Interactions ---")
formula_mod4 = ("AuC ~ (Age_std+Income_std + Anxiety_std + Amount_std + Education_std + Gender_c + Health_std)^2 + I(Age_std^2) + (1 + Amount_std|ID))")
idata_mod4 = betaMLM_pymc(formula_mod4, DL_dat4)
print(pymc_summary(idata_mod4))

## Supplementary Analyses: Depression and Distress Models

The paper notes that while anxiety was a significant predictor of discounting, depression and overall psychological distress were not (when controlling for age and income). This section presents the models that confirm these findings.

In [ ]:
# --- 6. SUPPLEMENTARY ANALYSES: DEPRESSION & DISTRESS ---

# --- Model S1: Test effect of Depression ---
print("--- Model S1: AuC ~ Age + Age^2 + Income + Depression ---")
idata_modS1 = betaMLM_pymc("AuC ~ Age_std + I(Age_std**2) + Income_std + Depression_std + (1|ID)", DL_dat3)
print(pymc_summary(idata_modS1, ['Intercept', 'Age_std', 'I(Age_std ** 2)', 'Income_std', 'Depression_std']))

# --- Model S3: Test effect of Distress ---
print("\n--- Model S3: AuC ~ Age + Age^2 + Income + Distress ---\n")
idata_modS3 = betaMLM_pymc("AuC ~ Age_std + I(Age_std**2) + Income_std + Distress_std + (1|ID)", DL_dat3)
print(pymc_summary(idata_modS3, ['Intercept', 'Age_std', 'I(Age_std ** 2)', 'Income_std', 'Distress_std']))